In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow as tf
from sklearn.metrics import mean_squared_error
pd.set_option('max_columns', 100)

Using TensorFlow backend.


In [2]:
import keras.backend as K
def rmse_keras(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred -y_true), axis=-1))

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(211, input_dim=211, kernel_initializer='normal', activation='relu'))
    model.add(Dense(6, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae',rmse_keras])
    return model

In [3]:
# fix random seed for reproducibility
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=1)

In [4]:
from sklearn.model_selection import KFold, StratifiedKFold

FEATS_EXCLUDED = ['first_active_month', 'target', 'card_id', 'outliers',
                  'hist_purchase_date_max', 'hist_purchase_date_min', 'hist_card_id_size',
                  'new_purchase_date_max', 'new_purchase_date_min', 'new_card_id_size',
                  'OOF_PRED', 'month_0']

def kfold_keras(train_df, test_df, num_folds, stratified = False, debug= False):
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=420)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=420)

    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in FEATS_EXCLUDED]

    # k-fold
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['outliers'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['target'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['target'].iloc[valid_idx]

        estimator.fit(train_x, train_y)
        
        oof_preds[valid_idx] = estimator.predict(valid_x)
        sub_preds += estimator.predict(test_df[feats]) / folds.n_splits

        print('Fold %2d RMSE : %.6f' % (n_fold + 1, rmse(valid_y, oof_preds[valid_idx])))
        pd.DataFrame(oof_preds).to_csv('oof_keras{}.csv'.format(n_fold))
        pd.DataFrame(sub_preds).to_csv('sub_keras{}.csv'.format(n_fold))

    pd.DataFrame(oof_preds).to_csv('oof_keras.csv')
    pd.DataFrame(sub_preds).to_csv('preds_keras.csv')
    if not debug:
        # save submission file
        test_df.loc[:,'target'] = sub_preds
        test_df = test_df.reset_index()
        test_df[['card_id', 'target']].to_csv(submission_file_name, index=False)

In [5]:
train_df = pd.read_csv('../working/train_df.csv')
test_df = pd.read_csv('../working/test_df.csv')

In [ ]:
kfold_keras(train_df, test_df, 5)

Starting LightGBM. Train shape: (201917, 221), test shape: (123623, 221)
Epoch 1/100
161533/161533 [==============================] - 58s 357us/step - loss: 15.0382 - mean_absolute_error: 1.5929 - rmse_keras: 1.5929
Epoch 2/100
161533/161533 [==============================] - 55s 340us/step - loss: 15.0334 - mean_absolute_error: 1.5976 - rmse_keras: 1.5976
Epoch 3/100
161533/161533 [==============================] - 48s 296us/step - loss: 15.0338 - mean_absolute_error: 1.5971 - rmse_keras: 1.5971
Epoch 4/100
161533/161533 [==============================] - 50s 310us/step - loss: 15.0338 - mean_absolute_error: 1.5965 - rmse_keras: 1.5965
Epoch 5/100
161533/161533 [==============================] - 48s 297us/step - loss: 15.0335 - mean_absolute_error: 1.5966 - rmse_keras: 1.5966
Epoch 6/100
161533/161533 [==============================] - 49s 301us/step - loss: 15.0337 - mean_absolute_error: 1.5973 - rmse_keras: 1.5973
Epoch 10/100
161533/161533 [==============================] - 49s 304

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



161533/161533 [==============================] - 50s 309us/step - loss: 15.0329 - mean_absolute_error: 1.5986 - rmse_keras: 1.5986
Epoch 19/100
161533/161533 [==============================] - 49s 304us/step - loss: 15.0338 - mean_absolute_error: 1.5961 - rmse_keras: 1.5961
Epoch 20/100
161533/161533 [==============================] - 49s 304us/step - loss: 15.0337 - mean_absolute_error: 1.5966 - rmse_keras: 1.5966
Epoch 21/100
151910/161533 [===========================>..] - ETA: 2s - loss: 14.9739 - mean_absolute_error: 1.5948 - rmse_keras: 1.5948

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



161533/161533 [==============================] - 53s 328us/step - loss: 15.0334 - mean_absolute_error: 1.5976 - rmse_keras: 1.5976
Epoch 33/100
161533/161533 [==============================] - 49s 302us/step - loss: 15.0337 - mean_absolute_error: 1.5968 - rmse_keras: 1.5968
Epoch 34/100
161533/161533 [==============================] - 49s 306us/step - loss: 15.0337 - mean_absolute_error: 1.5965 - rmse_keras: 1.5965
Epoch 35/100
103995/161533 [==================>...........] - ETA: 17s - loss: 15.0591 - mean_absolute_error: 1.5970 - rmse_keras: 1.5970

...

161533/161533 [==============================] - 55s 340us/step - loss: 14.8118 - mean_absolute_error: 1.5881 - rmse_keras: 1.5881
Epoch 58/100
161533/161533 [==============================] - 51s 316us/step - loss: 14.8119 - mean_absolute_error: 1.5881 - rmse_keras: 1.5881
Epoch 60/100
161533/161533 [==============================] - 48s 299us/step - loss: 14.8121 - mean_absolute_error: 1.5885 - rmse_keras: 1.5885
Epoch 61/100
161533/161533 [==============================] - 49s 301us/step - loss: 14.8115 - mean_absolute_error: 1.5876 - rmse_keras: 1.5876
Epoch 62/100
 80240/161533 [=============>................] - ETA: 24s - loss: 14.6435 - mean_absolute_error: 1.5858 - rmse_keras: 1.5858

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



161533/161533 [==============================] - 48s 299us/step - loss: 14.8113 - mean_absolute_error: 1.5890 - rmse_keras: 1.5890
Epoch 65/100
161533/161533 [==============================] - 48s 299us/step - loss: 14.8118 - mean_absolute_error: 1.5889 - rmse_keras: 1.5889
Epoch 67/100
161533/161533 [==============================] - 48s 299us/step - loss: 14.8117 - mean_absolute_error: 1.5885 - rmse_keras: 1.5885
Epoch 68/100
161533/161533 [==============================] - 49s 305us/step - loss: 14.8117 - mean_absolute_error: 1.5888 - rmse_keras: 1.5888
Epoch 69/100
134285/161533 [=======================>......] - ETA: 8s - loss: 14.7531 - mean_absolute_error: 1.5872 - rmse_keras: 1.5872

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



161533/161533 [==============================] - 48s 298us/step - loss: 14.8111 - mean_absolute_error: 1.5880 - rmse_keras: 1.5880
Epoch 71/100
161533/161533 [==============================] - 48s 300us/step - loss: 14.8116 - mean_absolute_error: 1.5892 - rmse_keras: 1.5892
Epoch 73/100
161533/161533 [==============================] - 48s 300us/step - loss: 14.8120 - mean_absolute_error: 1.5883 - rmse_keras: 1.5883
Epoch 74/100
161533/161533 [==============================] - 49s 302us/step - loss: 14.8117 - mean_absolute_error: 1.5885 - rmse_keras: 1.5885
Epoch 75/100
161533/161533 [==============================] - 49s 302us/step - loss: 14.8117 - mean_absolute_error: 1.5881 - rmse_keras: 1.5881
Epoch 76/100
 78980/161533 [=============>................] - ETA: 24s - loss: 14.8737 - mean_absolute_error: 1.5881 - rmse_keras: 1.5881